# Worms speech

TODO Explain what this notebook is about and why it exists.


## Data preprocessing


In [72]:
from pathlib import Path

import pandas as pd

raw = Path("data.txt").read_text()
rows = pd.Series(raw.splitlines())
rows.sample(n=10)

2295                   * ¡Date prisa! [Hurry up!] (Hurry)
2859                          * I felt a breeze. (Missed)
1873                                                     
237                       *Crap aim, by the way. (Missed)
809                           *Good work, men! (Flawless)
2042                  * haaa... Get down, fool! (Amazing)
2571         *Yeah, I'll have a bit of that. (Crate Drop)
1742    *Yeah I'm amazin', you won't fight me in the f...
1211          *Next question, for ten points... (Yes sir)
1184    *Oh no, you really should've banked before mak...
dtype: object

In [73]:
rows.head(n=20)

0     '''FSpeech'''/'''Voices'''/'''Speechbanks''' (...
1                                                      
2                                      == Speech Key ==
3        * Representation - When a worm start his turn.
4     *Hello / Wait - Plays when no controls have be...
5     *Come On Then - Plays when [[Turn Time]] is do...
6     *Hurry - Plays when [[Turn Time]] is down to 5...
7     *Watch This - Plays while a variable power wea...
8     *Panic - One of several voice files that play ...
9                                            **Run Away
10                                     **Nooo - Falling
11    **Grenade - Panic file specifically for enemy ...
12    **Take Cover - When an enemy calls an [[Air St...
13    *Stupid / Oi Nutter- Plays when a enemy worm d...
14    *Traitor - Plays when a worm damages teammate(...
15             *Hurt - Enemy Worm after being attacked.
16                    *Missed - Enemy missed an attack.
17                               *Oops - Attack 

In [74]:
rows.tail(n=20)

3005    *Langweilig. / Das war alles? (No attacks duri...
3006                          *Bye bye. / Oh nein. (Dies)
3007                               *Gewinner! (Team wins)
3008          *Zahnräder in Bewegung! (Crafting a weapon)
3009    *Spezialapparat eingesetzt! (Activating specia...
3010                                                     
3011                                           ==Trivia==
3012    *One of Soul Man's lines, "Like a stacks machi...
3013    *The quote "Say hello to my little friend!" fr...
3014    *The phrases "Shoryuken" and "Hadouken" are re...
3015    *In ''Worms 2: Armageddon'', the phrase "Fire ...
3016    *The phrase from the Horror soundbank "I'll te...
3017    *In Worms HD for the PS3, some speech banks ha...
3018                       **Double Oh Seven is named 17.
3019                      **Poor Rapper is called Rapper.
3020                        **Snitch is named Backstreet.
3021    *Worms saying the word "fatality" is likely a ...
3022    *In [[

In [75]:
# TODO Make less hacky :)
df = rows[20:-13].str.extract(r"\*(.+)\((.*)\)").dropna()
df.columns = ["speech_line", "speech_key"]
df.sample(n=10)


,speech_line,speech_key
2025,Funky chicken!,Amazing
2908,Grab your piece boys,Crate Drop
531,Hurry!,Hurry
764,I'll get you!,Hurt
1068,Impéccable! [Flawless!],Damage
2953,There was only 15 seconds left,Come on Then
1694,pudło... (missed...),Missed
2271,¡Espera y verás!,"""just you wait and see!"", I'll get you"
1303,Schnell!,Less than 5 seconds left
646,Alert.,Panic


In [76]:
# TODO Filter out speech keys with only one occurence.
df.groupby("speech_key").count().nlargest(25, columns="speech_line")

,speech_line
speech_key,
Death,56
Hurt,55
Coward,54
Hurry,51
Fatality,50
Panic,50
Stupid,49
Revenge,46
Boring,44


In [77]:
rows = df.rename(columns={"speech_key": "input", "speech_line": "output"}).to_dict(
    orient="records"
)

# TODO Send semistructured data directly instead of a string.
speechbank = ""
for d in rows:
    speechbank += f"input: {d['input']}\noutput: {d['output']}\n\n"

## Run text generation model to get speech lines


In [78]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="genai-nordics23sto-2304", location="us-central1")

parameters = {"temperature": 0.9, "max_output_tokens": 64, "top_p": 0.8, "top_k": 40}

model = TextGenerationModel.from_pretrained("text-bison")

# TODO Prepend personality.
worm_personality = """

"""

prompt = speechbank[:1000]

speech_key = "Missed"  # TODO Choice of speech key should be easier.

prompt += f"input: {speech_key}\noutput:"
response = model.predict(prompt, **parameters)

if response.text != "":
    print(f"{speech_key}: {response.text}")
else:
    print("No output :(")


Missed: 100% accuracy is impossible, so just enjoy the misses!


## Run text-to-speech (TTS) model to get audio


In [79]:
# TODO Use the TTS in Vertex AI instead.
!say "{response.text}"